In [1]:
import re
from collections import defaultdict
import json
import os
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
cwd = os.getcwd()
image_folder = os.path.join(cwd, 'images_disney')

In [3]:
if not os.path.exists(image_folder):
    os.mkdir(image_folder)

In [4]:
def get_image_link(filtered):
    for i in filtered:
        result = i.find('p', attrs = {'class': ''}).text
        #we only want movies directed by Hayao Miyazaki
        year_x = i.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text
        year = int(re.findall('\d+', year_x)[0])
        print(year)
        if year>1985:
            poster_link = i.find('div', attrs = {'class': 'lister-item-image float-left'}).find('img')['loadlate']
            images.append(poster_link)
            sub_link = i.find('h3', attrs = {'class': 'lister-item-header'}).find('a').get('href')
            l = 'https://www.imdb.com{sublink}?ref_=adv_li_i'.format(sublink = sub_link)
            links.append(l)
            r = requests.get(l)
            d = r.content
            f = BeautifulSoup(d).find('div', attrs = {'id': 'main_bottom'})
            try:
                storyline = f.find('div', attrs = {'class': 'inline canwrap'}).select('span')[0].text
                stories.append(storyline)
            except:
                continue
            img_links = f.find('div', attrs = {'class': 'mediastrip'}).find_all('a')
            for s in img_links:
                try:
                    img = s.select('img')[0]['loadlate']
                    images.append(img)
                except:
                    continue
        else:
            continue
    
    

In [5]:
def get_next_page(data):
    sub_page = BeautifulSoup(data).find('div', attrs = {'class': 'desc'}).find_all('a')[-1].get('href')
    next_page = 'https://www.imdb.com{subpage}&ref_=adv_nxt'.format(subpage = sub_page)
    return next_page

In [6]:
link = 'https://www.imdb.com/search/title/?companies=co0074039&sort=year,asc&ref_=adv_prv'
images = []
links = []
pages = [link]
stories = []
resp = requests.get(link)
data = resp.content
filtered = BeautifulSoup(data).find_all('div', attrs = {'class': 'lister-item mode-advanced'})
get_image_link(filtered)
while len(images) <= 200:
    link = get_next_page(data)
    if link not in pages:
        pages.append(link)
        print(link)
        resp = requests.get(link)
        data = resp.content
        filtered = BeautifulSoup(data).find_all('div', attrs = {'class': 'lister-item mode-advanced'})
        get_image_link(filtered)
    else:
        break


1928
1933
1937
1937
1940
1940
1941
1942
1942
1944
1946
1946
1947
1948
1949
1950
1951
1953
1955
1959
1961
1963
1967
1970
1973
1977
1977
1981
1982
1983
1985
1986
1988
1989
1989
1990
1991
1992
1994
1995
1995
1996
1997
1998
1999
1999
2000
2000
2000
2001
https://www.imdb.com/search/title/?companies=co0074039&sort=year,asc&start=51&ref_=adv_nxt
2002
2002
2003
2004
2005
2007
2007
2008
2008
2009
2009
2009
2009
2010
2010
2010
2010
2011
2011
2011
2011
2012
2012
2012
2012
2013
2013
2013
2014
2014
2014
2014
2015
2015
2015
2015
2016
2016
2016
2017
2017
2017
2017
2018
2018
2018
2018
2019
2019
2019


In [7]:
len(images)

410

In [17]:
len(stories)

64

In [18]:
for i in range(len(stories)):
    with open('storylines_disney.txt', 'a+') as fb:
        fb.write(stories[i])

In [11]:
for i in range(len(images)):
    store_as = image_folder+'/'+str(i)+'.jpg'
    img_data = requests.get(images[i]).content
    with open(store_as, 'wb') as fb:
        fb.write(img_data)